In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

# Charger les données
df = pd.read_csv("full_graph.csv", keep_default_na=False)

# Séparer les nœuds et les relations
nodes = df[df["_start"] == ""][["_id", "_labels", "name"]]
edges = df[df["_start"] != ""][["_start", "_end", "_type"]]

# Créer un graphe orienté avec états
G = nx.DiGraph()

# Ajouter les nœuds avec un état booléen 'visited'
for _, row in nodes.iterrows():
    G.add_node(row["_id"], 
               label=row["_labels"], 
               name=row["name"],
               visited=False)  # Attribut booléen ajouté

# Ajouter les relations
for _, row in edges.iterrows():
    G.add_edge(row["_start"], row["_end"], label=row["_type"])

# Fonction pour générer la couleur selon l'état
def get_node_color(node_data):
    return "#ff9999" if node_data["visited"] else "#99ccff"  # Rouge si visité, Bleu sinon

# Visualisation avec Matplotlib
plt.figure(figsize=(25, 20))
pos = nx.spring_layout(G, k=0.6, seed=42)

# Dessiner avec les couleurs d'état
node_colors = [get_node_color(G.nodes[node]) for node in G.nodes]
nx.draw_networkx_nodes(G, pos, node_size=1200, node_color=node_colors, alpha=0.9)
nx.draw_networkx_edges(G, pos, arrowstyle="->", arrowsize=25, edge_color="gray")

labels = {node: G.nodes[node]["name"] for node in G.nodes}
nx.draw_networkx_labels(G, pos, labels, font_size=9, font_weight="bold")

plt.title("Concept Graph with Node States", fontsize=16)
plt.axis("off")
plt.tight_layout()
plt.show()

# Version interactive avec PyVis (génère un fichier HTML)
net = Network(height="800px", width="100%", directed=True)
net.from_nx(G)

# Configuration visuelle
net.set_options("""
{
  "nodes": {
    "font": {"size": 14},
    "scaling": {"min": 20, "max": 40}
  },
  "edges": {
    "arrows": {"to": {"enabled": true}},
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -30000,
      "springLength": 200
    }
  }
}
""")

net.show("concept_graph.html")

ModuleNotFoundError: No module named 'pyvis'